In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tclab import TCLab
from package_PID_IMC import PID_IMC, IMC_PID

# --- Paramètres du Processus ---
Tg = 150.0
Tu = 6.0
Kp = 1.2
lambda_tuning = 50

# --- Calcul des paramètres PID optimaux avec IMC ---
Kp_PID, Ti, Td = IMC_PID(Tg, Tu, Kp, lambda_tuning)

# --- Initialisation du PID ---
Ts = 1.0
pid = PID_IMC(Kp_PID, Ti, Td, Ts)

# --- Initialisation du TCLab ---
lab = TCLab()
time.sleep(3)

# --- Expérience sur 100 itérations ---
SP = 50
temps, MVm, PVm = [], [], []

print("Expérience en cours...")

try:
    for t in range(100):
        start = time.time()

        PV = lab.T1
        MV = pid.compute(SP, PV)

        lab.Q1(MV)

        temps.append(t)
        MVm.append(MV)
        PVm.append(PV)

        elapsed = time.time() - start
        time.sleep(max(0, Ts - elapsed))

finally:
    lab.Q1(0)
    lab.close()

# --- Tracé des résultats ---
plt.figure(figsize=(10, 5))
plt.plot(temps, PVm, 'g-', label="PV (Température)")
plt.plot(temps, [SP] * len(temps), 'k--', label="Consigne")
plt.plot(temps, MVm, 'b-', label="Commande (MV)")
plt.xlabel("Temps [s]")
plt.ylabel("Température [°C]")
plt.title("PID Optimisé avec IMC sur TCLab")
plt.legend()
plt.grid()
plt.show()
